In [11]:

import pandas as pd

# Parquet faylni o'qish
df = pd.read_parquet("hf://datasets/Sanatbek/aspect-based-sentiment-analysis-uzbek/aspect-based-sentiment-analysis-uzbek.parquet")

# Tasodifiy aralashtirib, indeksni qayta tiklaymiz
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Ma'lumotni 80% train va 20% gold qilib bo'lish
train_size = int(0.8 * len(df))
train_df = df[:train_size]
gold_df = df[train_size:]

# JSON fayllarga yozish (orient='records' — har bir qator alohida JSON obyekt bo'ladi)
train_df.to_json("uzabsa_train.json", orient="records", force_ascii=False, lines=True)
gold_df.to_json("uzabsa_gold.json", orient="records", force_ascii=False, lines=True)


In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={
    "train":"uzabsa/uzabsa_gold.json",
    "test":"uzabsa/uzabsa_train.json"
})

train_data = dataset["train"]
test_data = dataset["test"]


c:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 2/2 [00:00<?, ?it/s]


Dataset json downloaded and prepared to C:/Users/User/.cache/huggingface/datasets/json/default-991ce515748bf468/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 39.90it/s]


In [8]:
dataset["train"]
print(train_data[0])


{'sentence_id': '1825#1', 'text': 'Turkiya fuqarosi sifatida aytishim kerakki, O‘zbekiston bo‘ylab safarim chog‘ida turk restoranida kechki ovqatdan juda xursand bo‘ldim.', 'aspect_terms': [{'from': 86, 'polarity': 'positive', 'term': 'restoranida', 'to': 97}], 'aspect_categories': [{'category': 'ovqat', 'polarity': 'positive'}]}


In [9]:
import xml.etree.ElementTree as ET
import os

def convert_to_semeval_xml(dataset, output_path, prefix="uz"):
    root = ET.Element("sentences")

    for idx, example in enumerate(dataset):
        sentence_el = ET.SubElement(root, "sentence", id=example.get("sentence_id", f"{prefix}#{idx}"))
        
        # <text>
        text_el = ET.SubElement(sentence_el, "text")
        text_el.text = example["text"]

        # <aspectTerms>
        aspect_terms_el = ET.SubElement(sentence_el, "aspectTerms")
        for term_info in example.get("aspect_terms", []):
            ET.SubElement(aspect_terms_el, "aspectTerm", {
                "term": term_info["term"],
                "polarity": term_info["polarity"],
                "from": str(term_info["from"]),
                "to": str(term_info["to"])
            })

        # <aspectCategories>
        aspect_categories_el = ET.SubElement(sentence_el, "aspectCategories")
        for cat_info in example.get("aspect_categories", []):
            ET.SubElement(aspect_categories_el, "aspectCategory", {
                "category": cat_info["category"],
                "polarity": cat_info["polarity"]
            })

    # XML faylga yozish
    tree = ET.ElementTree(root)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    tree.write(output_path, encoding="utf-8", xml_declaration=True)
    print(f"[✓] Saved: {output_path}")


In [10]:
convert_to_semeval_xml(train_data, "e:/uzgcae/output/train_semeval.xml", prefix="train")
convert_to_semeval_xml(test_data, "e:/uzgcae/output/test_semeval.xml", prefix="test")


[✓] Saved: e:/uzgcae/output/train_semeval.xml
[✓] Saved: e:/uzgcae/output/test_semeval.xml


KeyError: "Column aspect not in the dataset. Current columns in the dataset: ['sentence_id', 'text', 'aspect_terms', 'aspect_categories']"